# Preparing Documents

In [73]:
#dataset test 1
doc1 = "Sugar is bad to consume. My sister likes to have sugar, but not my father."
doc2 = "My father spends a lot of time driving my sister around to dance practice."
doc3 = "Doctors suggest that driving may cause increased stress and blood pressure."
doc4 = "Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better."
doc5 = "Health experts say that Sugar is not good for your lifestyle."
doc_test1="sugar is bad"
doc_test2="sugar is good"
#doc_complete = [doc1, doc2]
doc_complete = [doc1, doc2, doc3, doc4, doc5]
#doc_complete = [doc_test1, doc_test2]

In [2]:
# dataset test 2
import pandas as pd
xl = pd.ExcelFile("data.xlsx")
df = xl.parse("data")
df.columns = ['sujet', 'doc']
doc_complete = df['doc']
print(doc_complete)

0    This paper focuses on eight considerations whe...
1    Data Management and Visualization, Mastering S...
2    Learn SAS or Python programming, expand your k...
3    Data Analysis & Statistics Courses\nLearn abou...
4    Data Analysis is the process of systematically...
5    Roaming data analysis of business travellers a...
6    AUDIENCE DATAInterview with Jared Blank, SVP D...
7    Holographic analysis of Wi-Fi data generates 3...
Name: doc, dtype: object


# Cleaning and Preprocessing

In [3]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string
tokenizer = RegexpTokenizer(r'\w+')
stop = set(stopwords.words('english'))

exclude = set(string.punctuation) 
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
lemma = WordNetLemmatizer()
def clean(doc):
    # clean and tokenize document string
    raw = doc.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in stop]
    
    # remove punc_free from tokens
    punc_free = [ch for ch in stopped_tokens if ch not in exclude]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in punc_free]
    
    # lemm tokens
    normalized = [lemma.lemmatize(word) for word in punc_free]
   
    # add tokens to list
    return normalized
 
doc_clean = [clean(doc) for doc in doc_complete] 

# Preparing Document-Term Matrix

In [9]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

#print dic to ckeck
#for k, v in dictionary.iteritems():
    #print k, v

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

for i,doc in enumerate(doc_term_matrix):
    print "============ doc %s =======================" %(i)
    for d in doc:
        print "%s : %s"% (dictionary[d[0]], d[1])

============ doc 0 =======================
set : 1
help : 1
process : 1
intelligence : 1
focus : 1
analytics : 4
paper : 1
innovative : 1
83 : 1
skill : 1
extract : 1
derive : 1
business : 1
service : 1
giving : 1
data : 3
make : 1
since : 1
hadoop : 2
better : 1
needed : 1
memory : 1
leader : 1
around : 1
power : 1
importance : 1
big : 1
reader : 1
site : 1
management : 1
eight : 1
know : 1
consideration : 1
come : 1
optimize : 1
customer : 2
applying : 1
faster : 1
world : 1
1976 : 1
value : 1
preparation : 1
benefit : 1
000 : 1
learn : 1
sa : 3
decision : 1
software : 1
============ doc 1 =======================
process : 1
intelligence : 1
analytics : 17
skill : 2
business : 7
data : 48
hadoop : 1
big : 7
management : 4
customer : 1
decision : 2
software : 1
six : 1
interpretation : 1
è½ : 1
text : 4
executive : 1
spreadsheet : 1
course : 1
statistical : 1
mysql : 2
charting : 1
gi : 1
developer : 1
visualization : 6
scientistâ : 1
graph : 1
technique : 1
environment : 1
platform :

In [7]:
for i in doc_term_matrix:
    print i
print "================="
print doc_term_matrix[0]


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 4), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 3), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 3), (46, 1), (47, 1)]
[(2, 1), (3, 1), (5, 17), (9, 2), (12, 7), (15, 48), (18, 1), (26, 7), (29, 4), (35, 1), (46, 2), (47, 1), (48, 1), (49, 1), (50, 1), (51, 4), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 6), (61, 1), (62, 1), (63, 1), (64, 1), (65, 2), (66, 1), (67, 2), (68, 2), (69, 1), (70, 1), (71, 9), (72, 1), (73, 1), (74, 4), (75, 1), (76, 3), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 3), (87, 1), (88, 2), (89, 1), (90, 1), (91, 2), (92, 3), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 2), (99, 2), (1

# Running LDA Model

In [10]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
nb_topics=5
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=nb_topics, id2word = dictionary, passes=50)
print(ldamodel)

LdaModel(num_terms=1385, num_topics=5, decay=0.5, chunksize=2000)


# Results

In [11]:
# words by topic
print(ldamodel.print_topics(num_topics=nb_topics, num_words=3))


[(0, u'0.001*"data" + 0.001*"analysis" + 0.001*"method"'), (1, u'0.050*"data" + 0.015*"analytics" + 0.011*"platform"'), (2, u'0.027*"data" + 0.014*"business" + 0.014*"mobile"'), (3, u'0.017*"image" + 0.013*"wi" + 0.013*"fi"'), (4, u'0.034*"data" + 0.031*"analysis" + 0.011*"may"')]


In [12]:
#parametre : doc par term
#resut: topic with score by doc id
for i in range(8):
    print("doc : ",i)
    print(ldamodel.get_document_topics(doc_term_matrix[i]))

('doc : ', 0)
[(1, 0.98610729306501188)]
('doc : ', 1)
[(1, 0.99776642776703672)]
('doc : ', 2)
[(3, 0.99402989850249457)]
('doc : ', 3)
[(1, 0.21361191556701531), (2, 0.77426428050964691)]
('doc : ', 4)
[(4, 0.99939408286722797)]
('doc : ', 5)
[(2, 0.99613938530163515)]
('doc : ', 6)
[(1, 0.99910920547272331)]
('doc : ', 7)
[(3, 0.99828799958071301)]
